In [1]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=EBxxtXHsqvn57SEljnAM_KMCzmYVU0oNyTtjtjhxw8A&tc=dlXZujLq96jxcAa1s0NBXdFk3UD8430Kn-JRVKVDsYw&cc=ZwdqtHqeAbDkU9dWVrupv6SczyHfBYmr_DqhS361UAg

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhps0gSGZEe9wI3svEZ2yal76ygeBEzrWkYX_hgBOND8Tm1yShi8Hk

Successfully saved authorization token.


In [4]:
# this doesn't work with custom coord - unsure why ##


# This is the cloud masking function provided by GEE but adapted for use in Python.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)


# Define the geometry of the area for which you would like images

geom1 =[[33.8777, -13.4055], [33.8777, -13.3157], [33.9701, -13.3157], [33.9701, -13.4055]]

#test az coordinates but this fails
# geom1 = [[33.4268, -112.0068], [33.4268, -111.8452], [33.5615, -111.8452], [33.5615, -112.0068]]
geom = ee.Geometry.Polygon(geom1)

# Call collection of satellite images.
collection = (ee.ImageCollection("COPERNICUS/S2")
              # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
              .select(['B4', 'B3', 'B2', 'QA60'])
              # Filter for images within a given date range.
              .filter(ee.Filter.date('2017-01-01', '2017-03-31'))
              # Filter for images that overlap with the assigned geometry.
              .filterBounds(geom)
              # Filter for images that have less then 20% cloud coverage.
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
              # Apply cloud mask.
              .map(maskS2clouds)
             )

# Sort images in the collection by index (which is equivalent to sorting by date), 
# with the oldest images at the front of the collection.
# Convert collection into a single image mosaic where only images at the top of the collection are visible.
image = collection.sort('system:index', opt_ascending=False).mosaic()


# Assign visualization parameters to the image.
image = image.visualize(bands=['B4', 'B3', 'B2'],
                        min=[0.0, 0.0, 0.0],
                        max=[0.3, 0.3, 0.3]
                       )

# Assign export parameters.
task_config = {
    'region': geom.coordinates().getInfo(),
    'folder': 'Example_Folder_Name',
    'scale': 10,
    'crs': 'EPSG:4326',
    'description': 'customaz',
}

# Export Image
task = ee.batch.Export.image.toDrive(image, **task_config)
task.start()

In [6]:
!pip install earthengine-api
!pip install geopandas
import os
import sys
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.2 MB/s 
     |████████████████████████████████| 6.3 MB 46.7 MB/s 
     |████████████████████████████████| 16.7 MB 53.2 MB/s 


In [7]:
import ee
import pandas as pd
import numpy as np
import geopandas as gpd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
path = '/content/drive/MyDrive/tailings/fl_training2.csv'
df_bonus = pd.read_csv(path)

In [9]:
data = df_bonus.values

In [10]:
# Cloud masking function for Sentinel-2.
def maskS2clouds(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('QA60')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

In [11]:
# Use these bands for prediction.
# bands = ['B1', 'B2', 'B3', 'B4', 'B5','B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12']
bands = ['B4', 'B3', 'B2']

# Use Sentinel 2 surface reflectance data.
sentinel = ee.ImageCollection("COPERNICUS/S2")

In [12]:
# The image input data is cloud-masked median composite.
image = sentinel.filterDate('2018-01-01','2020-01-01').filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds).median().toFloat()

In [19]:
#code to extract coordinates
def offset(lat,lon,x,y):

	#Earth’s radius, sphere
	R=6378137

	#Coordinate offsets in radians
	dLat = x/R
	dLon = y/(R*math.cos(math.pi*lat/180))

	return lat + dLat * 180/math.pi, lon + dLon * 180/math.pi
 

def exportImage(data,size=0):
  # Loop the csv file.
  for d in range(data.shape[0]):
    x = data[d][0]
    y = data[d][1]
    
    llx , lly = offset(x,y,-size/2,-size/2)
    urx , ury = offset(x,y,size/2,	size/2)

    label = data[d][2]

    geom_raw = [[llx,lly], [llx,ury], [urx,ury], [urx,lly]]
    # geom_raw = [[33.8777, -13.4055], [33.8777, -13.3157], [33.9701, -13.3157], [33.9701, -13.4055]]

    task_config = {
	    'scale':  20 ,
	    'region': geom_raw
	    }
      
    name = label + str(d)
		# Create a task.
    task = ee.batch.Export.image(image, name, task_config)

		# Send the task to the earth engine.
    task.start()

In [20]:
data = df_bonus.values
exportImage(data,15000)